In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp
import sys
import scipy.io
sys.path.append("..")
import ODE_Dynamics as od
import Test_Likelihood as tl
import Positive_Symptom_fn as fn

In [2]:
data = scipy.io.loadmat('../data/test_data.mat')
x = tf.cast(data['data'],dtype = tf.float32)
test_data = np.reshape(
      x, x.shape)

In [3]:
test_data = tf.transpose(test_data)

In [4]:
vdyn_ode_fn = od.ViralDynamics

In [5]:
positive_fn = fn.proba_pos_sym(170306.4 * 1E-5).positive_fn
symptom_fn = fn.proba_pos_sym(170306.4 * 1E-5).symptom_fn

In [6]:
prob_s_ibar = 0.05

In [7]:
loglike = tl.loglik(test_data, vdyn_ode_fn, positive_fn, symptom_fn, prob_s_ibar, prob_fp=0.0, Epi_Model=od.SIR,
                 duration=31.0, Epi_cadence=0.5, Vir_cadence=0.0625)

In [8]:
mu_b, sigma_b = 5, 1
beta = np.random.normal(mu_b, sigma_b, 1)
L = 0.0025/beta
V0 = np.random.normal(1E3, 1E2, 1)
print(V0)
X0 = 1E6
Y0 = V0
par=tf.constant(np.array([[L,0.01,beta*1E-7,0.5,20,10, V0, X0, Y0]], dtype=np.float32))
vpar = par
pospar = par
sympar = par

[1058.40833403]


In [9]:
sample_size = 1000
k = 1
index = 1
mu_b, sigma_b = 5, 1
beta = np.random.normal(mu_b, sigma_b, 1)   #"rate at which virus infects host cells"
L = 0.0025/beta
par=np.array([[L,0.01,beta*1E-7,0.5,20.0,10.0]])
V0 = np.random.normal(1E3, 1E2, 1)
X0 = 1E6
Y0 = V0
init_state=(np.array([[V0,X0,Y0]], dtype=np.float32))

while index <= sample_size - 1:
    beta = np.random.normal(mu_b, sigma_b, 1)   #"rate at which virus infects host cells"
    L = 0.0025/beta
    par_new=np.array([[L,0.01,beta*1E-7,0.5,20.0,10.0]])
    par = np.concatenate((par, par_new), axis = 0)
    V0 = np.random.normal(1E3, 1E2, 1)
    X0 = 1E6
    Y0 = V0
    init_state_new=(np.array([[V0,X0,Y0]], dtype=np.float32))
    init_state = np.concatenate((init_state, init_state_new), 0)

    index +=1
        

vpar = tf.constant(par, dtype=tf.float32)
pospar = par
sympar = par
print(par.shape)

(1000, 6)


In [10]:
epipar = tf.constant(np.array([[1.8, 5E-8,0.1, 0.001,0.999]], dtype=np.float32))

In [11]:
ll = loglike.__call__(test_data,epipar, vpar, pospar, sympar)

(1, 31, 496)
printing p_given_s
tf.Tensor(
[0.07440393 0.078835   0.08340833 0.08811221 0.09293202 0.09785292
 0.10285678 0.10792528 0.11303802 0.11816609 0.12325604 0.12826389
 0.13315761 0.13791451 0.14252634 0.14701194 0.15138224 0.155636
 0.15976152 0.16373977 0.16754468 0.17114928 0.17452176 0.17763251
 0.18045217 0.18295367 0.18511325 0.1869108  0.18833022 0.18935977
 0.18999223], shape=(31,), dtype=float32)
printing likelihood
tf.Tensor(
[-585.77954  -231.9413   -303.30643  -173.9129   -101.80057  -301.7754
 -239.43524   -55.206596 -440.37656  -180.80652  -347.71805  -477.41125
 -458.17776  -199.24802  -193.57812  -232.27687  -176.88806  -215.47308
 -585.9016   -170.62727   -70.64346  -205.09425  -147.84085  -166.6677
  -56.30221  -394.93524  -439.22778  -427.1538   -575.7879   -190.78848
 -369.32574 ], shape=(31,), dtype=float32)


In [12]:
ll

<tf.Tensor: shape=(), dtype=float32, numpy=-8715.409>